In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
import lightgbm as lgb
import pickle
import os

In [2]:
flight_data = (
    pd.read_csv('final_dataset_cleaned_v1.csv')
)

In [3]:
flight_data.shape

(12658696, 29)

In [4]:
flight_data.head(n = 3)

YEAR  QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK     FL_DATE TAIL_NUM  \
0  2020        1      1             1            3  2020-01-01   N951WN   
1  2020        1      1             1            3  2020-01-01   N467WN   
2  2020        1      1             1            3  2020-01-01   N7885A   

  ORIGIN ORIGIN_CITY_NAME DEST  ...                 time  temp  dwpt  rhum  \
0    ONT      Ontario, CA  SFO  ...  2020-01-01 18:00:00  12.2  -7.3  25.0   
1    ONT      Ontario, CA  SFO  ...  2020-01-01 11:00:00   3.9  -3.4  59.0   
2    ONT      Ontario, CA  SJC  ...  2020-01-01 20:00:00  12.8  -7.3  24.0   

   prcp   wdir  wspd    pres coco target  
0   0.0  330.0  18.4  1017.3  3.0      1  
1   0.0  210.0   5.4  1017.6  2.0      0  
2   0.0  290.0  11.2  1015.8  2.0      0  

[3 rows x 29 columns]

In [5]:
flight_data['aircraft_type_new'] = [i.split('-')[0] for i in flight_data['Aircraft type']]

In [6]:
# list of numerical and categorical features to select 
l_numeric_cols = [ 'DEP_HOUR', 'temp', 'dwpt', 'rhum', 'prcp', 'wspd', 'pres', 'coco']
l_categorical_cols = ['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'aircraft_type_new']
target = "target"

In [7]:
# converting the categorical columns to categorical
for col in l_categorical_cols:
    flight_data[col] = flight_data[col].astype('category')

In [8]:
flight_data = flight_data[l_numeric_cols + l_categorical_cols + [target] + ['YEAR']]

In [9]:
train = flight_data[flight_data['YEAR'] == 2019]
test = flight_data[flight_data['YEAR'] == 2020]

In [10]:
del flight_data

In [11]:
from imblearn.under_sampling import RandomUnderSampler

undersample = RandomUnderSampler(sampling_strategy='majority', random_state=42)
X_res, y_res = undersample.fit_resample(train.drop(["YEAR","target"], axis=1).values,train[target])

In [12]:
del train

In [13]:
X_res1, y_res1 = undersample.fit_resample(X_res, y_res)
del X_res, y_res

In [14]:
X_res2, y_res2 = undersample.fit_resample(X_res1, y_res1)
del X_res1, y_res1

In [15]:
y_res2.value_counts()

0    215739
1    215739
2    215739
3    215739
Name: target, dtype: int64

In [16]:
train = pd.DataFrame(X_res2, columns=test.columns.drop(["YEAR","target"]))

In [17]:
train['target'] = y_res2

In [18]:
train.dtypes

DEP_HOUR             object
temp                 object
dwpt                 object
rhum                 object
prcp                 object
wspd                 object
pres                 object
coco                 object
QUARTER              object
MONTH                object
DAY_OF_MONTH         object
DAY_OF_WEEK          object
aircraft_type_new    object
target                int64
dtype: object

In [19]:
for col in l_categorical_cols:
    train[col] = train[col].astype('category')
    
for col in l_numeric_cols:
    train[col] = train[col].astype('float')

In [22]:
parameters = {
            'boosting_type': 'gbdt',
            'objective': 'multiclass',
            'num_class':4,
            "random_seed":42,
}

classifier = lgb.LGBMClassifier(**parameters)
classifier.fit(
    train.drop(["target"], axis=1),
     train[target],
    categorical_feature=l_categorical_cols,
    verbose=1
     )

C:\anaconda\envs\python_3_7_p\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


LGBMClassifier(num_class=4, objective='multiclass', random_seed=42)

In [23]:
path = "./models"
filename = "2019train_LGBM_undersample.mdl"

In [24]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(classifier, f, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
# predicting the test results 
y_pred = classifier.predict(test.drop(["YEAR","target"], axis=1))

In [26]:
print(confusion_matrix(test[target],y_pred))

[[2277165 1028557  536636  462025]
 [  96723   87407   45583   47345]
 [  25567   20943   17724   21232]
 [  14743   10979   10654   17950]]


In [27]:
print(classification_report(test[target],y_pred))

              precision    recall  f1-score   support

           0       0.94      0.53      0.68   4304383
           1       0.08      0.32      0.12    277058
           2       0.03      0.21      0.05     85466
           3       0.03      0.33      0.06     54326

    accuracy                           0.51   4721233
   macro avg       0.27      0.35      0.23   4721233
weighted avg       0.87      0.51      0.63   4721233



In [28]:
print("Testing  set score for model: %f" % classifier.score(test.drop(["YEAR","target"], axis=1)  , test[target] ))

Testing  set score for model: 0.508394


In [30]:
##### balanced class weight #####
parameters_b = {
            'boosting_type': 'gbdt',
            'objective': 'multiclass',
            'num_class':4,
            "random_seed":42,
            "class_weight":"balanced"
}

classifier_b = lgb.LGBMClassifier(**parameters_b)
classifier_b.fit(
    train.drop(["target"], axis=1),
     train[target],
    categorical_feature=l_categorical_cols,
    verbose=1
     )

C:\anaconda\envs\python_3_7_p\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


LGBMClassifier(class_weight='balanced', num_class=4, objective='multiclass',
               random_seed=42)

In [31]:
path = "./models"
filename = "2019train_LGBM_B_undersample.mdl"

In [32]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(classifier_b, f, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
# predicting the test results 
y_pred = classifier_b.predict(test.drop(["YEAR","target"], axis=1))

In [34]:
print(confusion_matrix(test[target],y_pred))

[[2277165 1028557  536636  462025]
 [  96723   87407   45583   47345]
 [  25567   20943   17724   21232]
 [  14743   10979   10654   17950]]


In [35]:
print(classification_report(test[target],y_pred))

              precision    recall  f1-score   support

           0       0.94      0.53      0.68   4304383
           1       0.08      0.32      0.12    277058
           2       0.03      0.21      0.05     85466
           3       0.03      0.33      0.06     54326

    accuracy                           0.51   4721233
   macro avg       0.27      0.35      0.23   4721233
weighted avg       0.87      0.51      0.63   4721233



In [36]:
print("Testing  set score for model: %f" % classifier_b.score(test.drop(["YEAR","target"], axis=1)  , test[target] ))

Testing  set score for model: 0.508394
